#1.load core modules

In [1]:
!pip install langchain
!pip install openai
!pip install pypdf
!pip install faiss-cpu
!pip install cohere
!pip install tiktoken
!pip install langchainhub
!pip install unstructured_pytesseract
!pip install unstructured_inference
!pip install pinecone-client

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.agents.agent_types import AgentType

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


#2.OPENAI/SERPAPI KEY Setting

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.environ["SERPAPI_API_KEY"] = "SERPAPI_API_KEY"
os.environ["PINECONE_API_KEY"] = "PINECONE_API_KEY"
os.environ["PINECONE_INDEX_NAME"] = "PINECONE_INDEX_NAME"

In [19]:
# LLM
from langchain.chat_models import ChatOpenAI

# ChatGPT 모델 지정
llm = ChatOpenAI(model_name="gpt-4-0613", temperature=0)
#llm = ChatOpenAI(model_name='gpt-3.5-turbo-1106',temperature=0)

In [20]:
embeddings = OpenAIEmbeddings()

#3.PINECONE Init

In [21]:
# initialize pinecone client and connect to pinecone index
pinecone.init(
        api_key=os.getenv("PINECONE_API_KEY"),
        environment="gcp-starter"
)

index_name = os.getenv("PINECONE_INDEX_NAME")
index = pinecone.Index(index_name) # connect to pinecone index

In [22]:
vectorstore = Pinecone.from_existing_index(index_name, embeddings)

corporate_analysis = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)


In [24]:
tools = [
    Tool(
        name = "Corporate Analysis",
        func=corporate_analysis.run,
        description="""
        Useful for when you need to answer questions about vetcor database.
        """
    )
]

In [25]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

memory = ConversationBufferMemory(memory_key="memory", return_messages=True)
#이부분 수정좀 해야함.뭘 수정해야하는지 찾아!!
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    'prefix': f'You are an expert in corporate analysis. You have access to the following tools:'
}

# initialize the LLM agent
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         agent_kwargs=agent_kwargs
                         )

In [26]:
agent.run("네가 가진 정보를 요약해줘 ")



> Entering new AgentExecutor chain...
이 질문에 대답하기 위해 원익큐엔씨에 대한 정보를 찾아야 한다.
Action: Corporate Analysis
Action Input: 원익큐엔씨
Observation: 원익큐엔씨는 중견기업으로 코스닥에 상장되어 있습니다. 이 회사는 기타 산업용 유리제품 제조업을 영위하고 있으며, 대표자는 백홍주입니다. 설립일은 2003년 11월 12일이고, 상장일은 같은 해 12월 12일입니다. 회사의 주소는 경북 구미시 옥계2공단로 117이며, 전화번호는 054-479-9500입니다. 종업원 수는 1,143명입니다. 기업평가 등급은 BBB+로, 상거래를 위한 신용능력이 양호하나, 경제여건 및 환경악화에 따라 거래안정성 저하 가능성이 있는 기업으로 평가되었습니다.
Thought:이제 원익큐엔씨에 대한 정보를 알았다.
Final Answer: 원익큐엔씨는 기타 산업용 유리제품 제조업을 영위하는 중견기업입니다. 이 회사는 코스닥에 상장되어 있으며, 대표자는 백홍주입니다. 회사의 주소는 경북 구미시 옥계2공단로 117이며, 전화번호는 054-479-9500입니다. 종업원 수는 1,143명이며, 기업평가 등급은 BBB+입니다.

> Finished chain.


'원익큐엔씨는 기타 산업용 유리제품 제조업을 영위하는 중견기업입니다. 이 회사는 코스닥에 상장되어 있으며, 대표자는 백홍주입니다. 회사의 주소는 경북 구미시 옥계2공단로 117이며, 전화번호는 054-479-9500입니다. 종업원 수는 1,143명이며, 기업평가 등급은 BBB+입니다.'